In [ ]:
import zipfile
import os

# Paths to your zip files
zip_paths = [
    "/content/augmented_images.zip",
    "/content/augmented_images(white throated kingfisher).zip",
    "/content/augmented_images(red vented bulbul).zip"
]

# Unzipping function
def unzip_files(zip_paths, extract_to_base):
    for zip_path in zip_paths:
        # Get the folder name based on the zip file name
        folder_name = os.path.splitext(os.path.basename(zip_path))[0]
        extract_to = os.path.join(extract_to_base, folder_name)

        os.makedirs(extract_to, exist_ok=True)
        print(f"Unzipping {zip_path} to {extract_to}...")

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Finished unzipping {zip_path} to {extract_to}")

# Unzip all files to separate directories for each bird
extracted_base_dir = "/content/extracted_images"
unzip_files(zip_paths, extracted_base_dir)


Unzipping /content/augmented_images.zip to /content/extracted_images/augmented_images...
Finished unzipping /content/augmented_images.zip to /content/extracted_images/augmented_images
Unzipping /content/augmented_images(white throated kingfisher).zip to /content/extracted_images/augmented_images(white throated kingfisher)...
Finished unzipping /content/augmented_images(white throated kingfisher).zip to /content/extracted_images/augmented_images(white throated kingfisher)
Unzipping /content/augmented_images(red vented bulbul).zip to /content/extracted_images/augmented_images(red vented bulbul)...
Finished unzipping /content/augmented_images(red vented bulbul).zip to /content/extracted_images/augmented_images(red vented bulbul)


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Function to split and organize datasets
def split_and_organize_datasets(base_extracted_dir, split_base_dir, test_size=0.2, val_size=0.2):
    # Clear or create split directories
    if os.path.exists(split_base_dir):
        shutil.rmtree(split_base_dir)
    os.makedirs(split_base_dir, exist_ok=True)

    # Create subdirectories for train, validation, and test
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(split_base_dir, split), exist_ok=True)

    # Loop through each bird folder and split its images
    for bird_folder in os.listdir(base_extracted_dir):
        bird_dir = os.path.join(base_extracted_dir, bird_folder)
        if not os.path.isdir(bird_dir):
            continue

        print(f"Processing {bird_folder}...")

        # Get all image files in the current bird folder
        images = os.listdir(bird_dir)
        images = [img for img in images if img.endswith(('.jpg', '.jpeg', '.png'))]  # Filter valid image files

        if len(images) == 0:
            print(f"No valid image files found in {bird_folder}")
            continue

        # Split the images into train, test, and validation sets
        train_imgs, test_imgs = train_test_split(images, test_size=0.2, random_state=42)
        train_imgs, val_imgs = train_test_split(train_imgs, test_size=0.2, random_state=42)

        # Copy images to the corresponding directories
        for split, imgs in zip(['train', 'val', 'test'], [train_imgs, val_imgs, test_imgs]):
            split_dir = os.path.join(split_base_dir, split, bird_folder)
            os.makedirs(split_dir, exist_ok=True)
            for img in imgs:
                shutil.copy(os.path.join(bird_dir, img), os.path.join(split_dir, img))

# Path to your extracted dataset directory
base_extracted_dir = "/content/extracted_images"
split_base_dir = "/content/split_datasets"
split_and_organize_datasets(base_extracted_dir, split_base_dir)


Processing augmented_images(red vented bulbul)...
Processing augmented_images(white throated kingfisher)...
Processing augmented_images...


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Image size and batch size settings
img_size = (224, 224)
batch_size = 32

# Create the ImageDataGenerator objects for train, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Define the directories for train, validation, and test
train_dir = os.path.join("/content/split_datasets", 'train')
val_dir = os.path.join("/content/split_datasets", 'val')
test_dir = os.path.join("/content/split_datasets", 'test')

# Create data generators for train, validation, and test
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 1920 images belonging to 3 classes.
Found 480 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [ ]:



import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the MobileNetV2 model with pre-trained weights
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Unfreeze the last 15 layers for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Add L2 regularization to base model layers
for layer in base_model.layers[-15:]:
    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer = tf.keras.regularizers.l2(0.01)

# Create the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)),  # Increased neurons
    Dropout(0.2),  # Reduced dropout
    Dense(train_gen.num_classes, activation='softmax')
])

# Compile the model with label smoothing
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Increased learning rate
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.01),  # Label smoothing to reduce overconfidence
    metrics=['accuracy']
)



In [ ]:
# Train the model
epochs = 10
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    verbose=1
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.5564 - loss: 5.5897 - val_accuracy: 0.8292 - val_loss: 5.2099
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.8428 - loss: 5.1281 - val_accuracy: 0.8813 - val_loss: 5.0171
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 125s 2s/step - accuracy: 0.9075 - loss: 4.9301 - val_accuracy: 0.9125 - val_loss: 4.8886
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.9144 - loss: 4.8252 - val_accuracy: 0.9125 - val_loss: 4.7935
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 125s 2s/step - accuracy: 0.9357 - loss: 4.7084 - val_accuracy: 0.9208 - val_loss: 4.7027
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 120s 2s/step - accuracy: 0.9517 - loss: 4.6219 - val_accuracy: 0.9292 - val_loss: 4.6279
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 152s 3s/step - accuracy: 0.9579 - loss: 4.5558 - val_accuracy: 0.9396 - val_loss: 4.5520
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 124s 2s/step - accuracy: 0.9587 - loss: 4.4792 - val_accuracy: 0.9542 - v

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Calculate the average training accuracy across all epochs
average_training_accuracy = sum(history.history['accuracy']) / len(history.history['accuracy'])

# Print the result
print(f"Average Training Accuracy: {average_training_accuracy:.4f}")

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(test_gen, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Generate classification report
y_true = test_gen.classes  # Ground truth labels
y_pred = np.argmax(model.predict(test_gen), axis=-1)  # Predicted labels
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_gen.class_indices.keys()))


Average Training Accuracy: 0.9081
19/19 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.9730 - loss: 4.3031
Test Accuracy: 94.67%
19/19 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step
Classification Report:
                                             precision    recall  f1-score   support

                           augmented_images       0.89      0.99      0.94       200
        augmented_images(red vented bulbul)       0.98      0.95      0.97       200
augmented_images(white throated kingfisher)       0.97      0.90      0.94       200

                                   accuracy                           0.95       600
                                  macro avg       0.95      0.95      0.95       600
                               weighted avg       0.95      0.95      0.95       600



In [ ]:
model.save("/content/bird_classification_model.h5")  # Saves the model to the specified path
from google.colab import files
files.download("/content/bird_classification_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model (if not already loaded)
# model = tf.keras.models.load_model('path_to_your_trained_model')

# Path to the image you want to classify
image_path = '/content/download (1).jpeg'

# Load the image and resize it to the target size used during training (224x224 in your case)
img = image.load_img(image_path, target_size=(224, 224))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Rescale the image (normalize it to the range [0, 1])
img_array = img_array / 255.0

# Expand the dimensions to match the input shape of the model (batch size, height, width, channels)
img_array = np.expand_dims(img_array, axis=0)

# Make the prediction
predictions = model.predict(img_array)

# Get the class with the highest predicted probability
predicted_class = np.argmax(predictions, axis=-1)

# Get the class label from the class indices
class_labels = list(train_gen.class_indices.keys())
predicted_class_label = class_labels[predicted_class[0]]

print(f"Predicted Class: {predicted_class_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Predicted Class: augmented_images(red vented bulbul)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model (if not already loaded)
# model = tf.keras.models.load_model('path_to_your_trained_model')

# Path to the image you want to classify
image_path = '/content/download (2).jpeg'

# Load the image and resize it to the target size used during training (224x224 in your case)
img = image.load_img(image_path, target_size=(224, 224))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Rescale the image (normalize it to the range [0, 1])
img_array = img_array / 255.0

# Expand the dimensions to match the input shape of the model (batch size, height, width, channels)
img_array = np.expand_dims(img_array, axis=0)

# Make the prediction
predictions = model.predict(img_array)

# Get the class with the highest predicted probability
predicted_class = np.argmax(predictions, axis=-1)

# Get the probability of the predicted class
predicted_probability = predictions[0][predicted_class[0]]

# Define a threshold for confidence (e.g., 0.7 or 70%)
confidence_threshold = 0.7

if predicted_probability < confidence_threshold:
    print("Cannot identify with high confidence.")
else:
    # Get the class label from the class indices
    class_labels = list(train_gen.class_indices.keys())
    predicted_class_label = class_labels[predicted_class[0]]
    print(f"Predicted Class: {predicted_class_label} with confidence: {predicted_probability:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Cannot identify with high confidence.
